# A2C Agent 

In [4]:
import gym
from a2c import A2CAgent 
import time
import numpy as np

# Create Gym environment
a2c_env = "CartPole-v1"
env = gym.make(a2c_env)

# Check agent class for initialization parameters and initialize agent
if a2c_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-3

agent = A2CAgent(env, gamma, lr)

# Define training parameters
max_episodes = 500
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    trajectory = []
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        trajectory.append((state, action, reward, next_state, done))
        episode_reward += reward  
        if done or step == max_steps:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break
        state = next_state
    agent.update(trajectory, 0)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
a2c_rewards = episode_rewards
a2c_runtime = run_time

Episode 0: 25.0
Episode 1: 17.0
Episode 2: 28.0
Episode 3: 17.0
Episode 4: 35.0
Episode 5: 18.0
Episode 6: 15.0
Episode 7: 72.0
Episode 8: 18.0
Episode 9: 82.0
Episode 10: 18.0
Episode 11: 27.0
Episode 12: 48.0
Episode 13: 161.0
Episode 14: 44.0
Episode 15: 72.0
Episode 16: 35.0
Episode 17: 57.0
Episode 18: 60.0
Episode 19: 15.0
Episode 20: 97.0
Episode 21: 30.0
Episode 22: 44.0
Episode 23: 49.0
Episode 24: 17.0
Episode 25: 18.0
Episode 26: 58.0
Episode 27: 31.0
Episode 28: 40.0
Episode 29: 51.0
Episode 30: 54.0
Episode 31: 45.0
Episode 32: 38.0
Episode 33: 33.0
Episode 34: 86.0
Episode 35: 70.0
Episode 36: 52.0
Episode 37: 80.0
Episode 38: 39.0
Episode 39: 63.0
Episode 40: 26.0
Episode 41: 62.0
Episode 42: 45.0
Episode 43: 67.0
Episode 44: 54.0
Episode 45: 19.0
Episode 46: 59.0
Episode 47: 53.0
Episode 48: 87.0
Episode 49: 118.0
Episode 50: 55.0
Episode 51: 84.0
Episode 52: 40.0
Episode 53: 38.0
Episode 54: 62.0
Episode 55: 60.0
Episode 56: 168.0
Episode 57: 33.0
Episode 58: 198.0
Epi

Episode 441: 88.0
Episode 442: 212.0
Episode 443: 177.0
Episode 444: 94.0
Episode 445: 301.0
Episode 446: 351.0
Episode 447: 106.0
Episode 448: 85.0
Episode 449: 500.0
Episode 450: 500.0
Episode 451: 104.0
Episode 452: 90.0
Episode 453: 351.0
Episode 454: 375.0
Episode 455: 393.0
Episode 456: 371.0
Episode 457: 436.0
Episode 458: 187.0
Episode 459: 245.0
Episode 460: 114.0
Episode 461: 242.0
Episode 462: 249.0
Episode 463: 251.0
Episode 464: 393.0
Episode 465: 263.0
Episode 466: 107.0
Episode 467: 245.0
Episode 468: 115.0
Episode 469: 242.0
Episode 470: 130.0
Episode 471: 237.0
Episode 472: 140.0
Episode 473: 232.0
Episode 474: 120.0
Episode 475: 138.0
Episode 476: 220.0
Episode 477: 127.0
Episode 478: 136.0
Episode 479: 112.0
Episode 480: 119.0
Episode 481: 116.0
Episode 482: 123.0
Episode 483: 110.0
Episode 484: 109.0
Episode 485: 122.0
Episode 486: 113.0
Episode 487: 126.0
Episode 488: 117.0
Episode 489: 114.0
Episode 490: 110.0
Episode 491: 107.0
Episode 492: 117.0
Episode 493: 100

In [5]:
name = './log_files/a2c/' + a2c_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((a2c_runtime, a2c_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

# DR TRPO Agent (KL) 

In [31]:
import gym
from a2c_dr_trpo import DRTRPOAgent 
import time
import numpy as np

# Create Gym environment
kl_env = "CartPole-v1"
env = gym.make(kl_env)

# Check agent class for initialization parameters and initialize agent
if kl_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-2
    beta = 0.8

agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 500
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    first_state = env.reset()
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps-1:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
    
    policy_loss = agent.compute_policy_loss_kl(state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    avg_episode_reward = episode_reward/env.action_space.n
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_kl_rewards = episode_rewards
dr_trpo_kl_runtime = run_time

Episode 0: 33.5
Episode 1: 13.5
Episode 2: 15.0
Episode 3: 13.0
Episode 4: 16.5
Episode 5: 23.5
Episode 6: 31.0
Episode 7: 23.0
Episode 8: 25.0
Episode 9: 29.0
Episode 10: 32.5
Episode 11: 20.5
Episode 12: 21.5
Episode 13: 42.0
Episode 14: 40.5
Episode 15: 35.5
Episode 16: 28.5
Episode 17: 36.5
Episode 18: 38.0
Episode 19: 41.0
Episode 20: 44.0
Episode 21: 45.5
Episode 22: 50.5
Episode 23: 78.5
Episode 24: 99.5
Episode 25: 63.5
Episode 26: 54.0
Episode 27: 138.0
Episode 28: 100.5
Episode 29: 94.0
Episode 30: 134.5
Episode 31: 80.5
Episode 32: 99.0
Episode 33: 162.5
Episode 34: 125.5
Episode 35: 91.5
Episode 36: 103.0
Episode 37: 143.0
Episode 38: 213.5
Episode 39: 291.0
Episode 40: 500.0
Episode 41: 500.0
Episode 42: 440.5
Episode 43: 449.5
Episode 44: 431.5
Episode 45: 403.5
Episode 46: 492.5
Episode 47: 447.5
Episode 48: 500.0
Episode 49: 430.5
Episode 50: 349.0
Episode 51: 500.0
Episode 52: 482.5
Episode 53: 302.0
Episode 54: 387.0
Episode 55: 381.5
Episode 56: 452.5
Episode 57: 387

Episode 440: 377.0
Episode 441: 384.0
Episode 442: 242.0
Episode 443: 335.5
Episode 444: 372.0
Episode 445: 340.5
Episode 446: 381.5
Episode 447: 400.0
Episode 448: 439.5
Episode 449: 333.5
Episode 450: 374.5
Episode 451: 340.0
Episode 452: 416.0
Episode 453: 315.0
Episode 454: 492.0
Episode 455: 335.0
Episode 456: 391.0
Episode 457: 346.5
Episode 458: 500.0
Episode 459: 395.5
Episode 460: 283.0
Episode 461: 463.0
Episode 462: 500.0
Episode 463: 274.0
Episode 464: 352.5
Episode 465: 463.0
Episode 466: 401.5
Episode 467: 362.0
Episode 468: 359.0
Episode 469: 372.0
Episode 470: 363.0
Episode 471: 373.0
Episode 472: 199.5
Episode 473: 328.0
Episode 474: 311.5
Episode 475: 347.5
Episode 476: 421.0
Episode 477: 346.5
Episode 478: 330.5
Episode 479: 371.5
Episode 480: 431.5
Episode 481: 241.5
Episode 482: 350.0
Episode 483: 371.5
Episode 484: 341.5
Episode 485: 310.5
Episode 486: 380.0
Episode 487: 300.5
Episode 488: 490.0
Episode 489: 411.5
Episode 490: 309.5
Episode 491: 299.5
Episode 492:

In [32]:
name = './log_files/dr_trpo_kl/' + kl_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_kl_runtime, dr_trpo_kl_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

# DR TRPO Agent (Wasserstein)

In [12]:
import gym
from a2c_dr_trpo import DRTRPOAgent 
import time
import numpy as np

wass_env = "CartPole-v1"
# Create Gym environment
env = gym.make(wass_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-2
    
agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 500
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    first_state = env.reset()
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps-1:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
    
    total_adv_diff += abs(state_adv[1] - state_adv[0])
    beta = total_adv_diff/episode
    beta += np.random.random()*0.3-0.1
    policy_loss = agent.compute_policy_loss_wass(state, state_adv, beta)

    
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    avg_episode_reward = episode_reward/env.action_space.n
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_wass_rewards = episode_rewards
dr_trpo_wass_runtime = run_time

Episode 0: 29.5
Episode 1: 15.0
Episode 2: 10.0
Episode 3: 13.5
Episode 4: 12.0
Episode 5: 12.0
Episode 6: 12.0
Episode 7: 12.5
Episode 8: 10.0
Episode 9: 11.5
Episode 10: 11.0
Episode 11: 10.0
Episode 12: 10.0
Episode 13: 10.0
Episode 14: 9.5
Episode 15: 10.5
Episode 16: 12.0
Episode 17: 11.5
Episode 18: 11.0
Episode 19: 10.5
Episode 20: 12.5
Episode 21: 17.5
Episode 22: 12.5
Episode 23: 19.5
Episode 24: 20.0
Episode 25: 61.0
Episode 26: 44.5
Episode 27: 48.5
Episode 28: 41.0
Episode 29: 41.0
Episode 30: 59.0
Episode 31: 50.5
Episode 32: 71.0
Episode 33: 58.5
Episode 34: 65.5
Episode 35: 78.5
Episode 36: 54.0
Episode 37: 60.0
Episode 38: 60.5
Episode 39: 56.0
Episode 40: 69.0
Episode 41: 66.0
Episode 42: 87.5
Episode 43: 105.5
Episode 44: 106.0
Episode 45: 104.0
Episode 46: 112.5
Episode 47: 115.0
Episode 48: 139.5
Episode 49: 104.5
Episode 50: 104.5
Episode 51: 133.5
Episode 52: 102.0
Episode 53: 88.0
Episode 54: 145.0
Episode 55: 131.5
Episode 56: 138.5
Episode 57: 99.5
Episode 58: 

Episode 443: 221.5
Episode 444: 236.0
Episode 445: 164.0
Episode 446: 187.0
Episode 447: 303.5
Episode 448: 233.5
Episode 449: 343.5
Episode 450: 216.5
Episode 451: 284.0
Episode 452: 185.5
Episode 453: 270.5
Episode 454: 184.0
Episode 455: 209.0
Episode 456: 263.5
Episode 457: 206.0
Episode 458: 302.5
Episode 459: 245.5
Episode 460: 236.5
Episode 461: 282.5
Episode 462: 240.5
Episode 463: 197.5
Episode 464: 243.0
Episode 465: 164.5
Episode 466: 194.0
Episode 467: 234.0
Episode 468: 228.5
Episode 469: 181.5
Episode 470: 280.0
Episode 471: 247.0
Episode 472: 212.0
Episode 473: 160.5
Episode 474: 186.5
Episode 475: 217.5
Episode 476: 194.0
Episode 477: 175.5
Episode 478: 234.5
Episode 479: 252.5
Episode 480: 169.0
Episode 481: 186.0
Episode 482: 230.0
Episode 483: 262.0
Episode 484: 239.0
Episode 485: 311.5
Episode 486: 188.0
Episode 487: 235.5
Episode 488: 196.0
Episode 489: 295.0
Episode 490: 227.5
Episode 491: 175.0
Episode 492: 309.0
Episode 493: 224.5
Episode 494: 208.0
Episode 495:

In [13]:
name = './log_files/dr_trpo_wass/' + wass_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_wass_runtime, dr_trpo_wass_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')